In [18]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

In [19]:
'''
He等人（2016）在研究非常深的计算机视觉模型时考虑了这个问题。他们提出的残差网络（ResNet）的核心思想是，每个额外的层都应该更容易地将恒等函数作为其元素之一。
这些考虑相当深刻，但它们导致了一个非常简单的解决方案：残差块。有了它，ResNet赢得了2015年ImageNet大规模视觉识别挑战赛。
这种设计对如何构建深度神经网络产生了深远的影响。例如，残差块已被添加到循环网络（Kim等人，2017; Prakash等人，2016）。
同样，Transformer（Vaswani等人，2017）使用它们有效地堆叠多层网络。它还用于图神经网络（Kipf和Welling，2016），
作为一个基本概念，它已被广泛用于计算机视觉（Redmon和Farhadi，2018; Ren等人，2015）。需要注意的是，残差网络在时间上早于高速公路网络（Srivastava等人，2015），尽管它们分享了一些动机，但没有围绕恒等函数的优雅参数化。

简而言之，残差网络（ResNet）的关键思想是通过引入残差块来增加网络的深度，而不损害其性能。
残差块的设计使得每个额外的层都可以更容易地包含恒等函数，从而有助于减小训练误差。
自从提出以来，残差块已经被广泛应用于各种深度学习模型，如循环网络、Transformer、图神经网络和计算机视觉领域的各种模型。这种设计对深度神经网络的构建产生了重要影响，并为许多其他领域的研究提供了灵感。
'''


class Residual(nn.Module):  # @save
    """The Residual block of ResNet models."""

    def __init__(self, num_channels, use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1,
                                   stride=strides)
        self.conv2 = nn.LazyConv2d(num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.LazyConv2d(num_channels, kernel_size=1,
                                       stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.LazyBatchNorm2d()
        self.bn2 = nn.LazyBatchNorm2d()

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

In [20]:
blk = Residual(3)
X = torch.randn(4, 3, 6, 6)
blk(X).shape

torch.Size([4, 3, 6, 6])

In [21]:
blk = Residual(6, use_1x1conv=True, strides=2)
blk(X).shape

torch.Size([4, 6, 3, 3])

In [22]:
class ResNet(d2l.Classifier):
    def b1(self):
        return nn.Sequential(
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            nn.LazyBatchNorm2d(), nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [23]:
@d2l.add_to_class(ResNet)
def block(self, num_residuals, num_channels, first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(num_channels, use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels))
    return nn.Sequential(*blk)

In [24]:
@d2l.add_to_class(ResNet)
def __init__(self, arch, lr=0.1, num_classes=10):
    super(ResNet, self).__init__()
    self.save_hyperparameters()
    self.net = nn.Sequential(self.b1())
    for i, b in enumerate(arch):
        self.net.add_module(f'b{i+2}', self.block(*b, first_block=(i == 0)))
    self.net.add_module('last', nn.Sequential(
        nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(),
        nn.LazyLinear(num_classes)))
    self.net.apply(d2l.init_cnn)

In [25]:
class ResNet18(ResNet):
    def __init__(self, lr=0.1, num_classes=10):
        super().__init__(((2, 64), (2, 128), (2, 256), (2, 512)), lr, num_classes)


ResNet18().layer_summary((1, 1, 96, 96))

Sequential output shape:	 torch.Size([1, 64, 24, 24])
Sequential output shape:	 torch.Size([1, 64, 24, 24])
Sequential output shape:	 torch.Size([1, 128, 12, 12])
Sequential output shape:	 torch.Size([1, 256, 6, 6])
Sequential output shape:	 torch.Size([1, 512, 3, 3])
Sequential output shape:	 torch.Size([1, 10])


In [27]:
model = ResNet18(lr=0.01)
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
data = d2l.FashionMNIST(batch_size=128, resize=(96, 96))
model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
trainer.fit(model, data)

/data2/miniconda3/envs/d2l-pytorch/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


RuntimeError: GET was unable to find an engine to execute this computation

In [29]:
'''8_1_deep_convolutional_nerual_networks_alexnet.ipynb
将一个卷积从C_in到C_out通道分解为一个C_in/groups大小的组，生成C_out/groups大小的输出，这称为分组卷积。计算成本（成比例地）从C_in * C_out * k^2减少到C_in * C_out * k^2 / groups，即它快groups倍。更好的是，生成输出所需的参数数量也从C_in * C_out矩阵减少到groups个大小为C_in/groups * C_out/groups的较小矩阵，再次减少groups倍。在接下来的讨论中，我们假设C_in和C_out都可以被groups整除。

这种设计的唯一挑战是groups之间没有信息交换。如图8.6.5所示的ResNeXt块以两种方式进行修正：组卷积与1x1卷积夹在一起。第二个1x1卷积兼具改变通道数量的功能。好处是我们只需要为k^2核付出groups倍的成本，而对于1x1核，成本是C_in/groups * C_out/groups。类似于第8.6.2节中残差块的实现，残差连接被1x1卷积替换（从而泛化）。

图8.6.5中的右侧图形提供了对生成的网络块的更简洁的总结。它还将在第8.8节中通用现代CNN的设计中起重要作用。请注意，分组卷积的概念可以追溯到AlexNet（Krizhevsky等人，2012）的实现。在将网络分布到具有有限内存的两个GPU上时，实现将每个GPU视为自己的通道，没有不良影响。

以下ResNeXtBlock类的实现将groups（C_in/groups）作为参数，带有bot_channels（C_in/groups * C_out/groups）中间（瓶颈）通道。最后，当我们需要减小表示的高度和宽度时，我们通过设置use_1x1conv=True，strides=2添加步幅为2的1x1卷积。
''' 

class ResNeXtBlock(nn.Module):  # @save
    """The ResNeXt block."""

    def __init__(self, num_channels, groups, bot_mul, use_1x1conv=False,
                 strides=1):
        super().__init__()
        bot_channels = int(round(num_channels * bot_mul))
        self.conv1 = nn.LazyConv2d(bot_channels, kernel_size=1, stride=1)
        self.conv2 = nn.LazyConv2d(bot_channels, kernel_size=3,
                                   stride=strides, padding=1,
                                   groups=bot_channels//groups)
        self.conv3 = nn.LazyConv2d(num_channels, kernel_size=1, stride=1)
        self.bn1 = nn.LazyBatchNorm2d()
        self.bn2 = nn.LazyBatchNorm2d()
        self.bn3 = nn.LazyBatchNorm2d()
        if use_1x1conv:
            self.conv4 = nn.LazyConv2d(num_channels, kernel_size=1,
                                       stride=strides)
            self.bn4 = nn.LazyBatchNorm2d()
        else:
            self.conv4 = None

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = F.relu(self.bn2(self.conv2(Y)))
        Y = self.bn3(self.conv3(Y))
        if self.conv4:
            X = self.bn4(self.conv4(X))
        return F.relu(Y + X)

In [30]:
blk = ResNeXtBlock(32, 16, 1)
X = torch.randn(4, 32, 96, 96)
blk(X).shape

torch.Size([4, 32, 96, 96])

'''8_1_deep_convolutional_nerual_networks_alexnet.ipynb
嵌套函数类是可取的，因为它们允许我们在增加容量时获得严格更强大的功能类，而不是也有微妙的不同功能类。实现这一点的一种方法是让额外的层简单地将输入传递到输出。残差连接允许这样做。因此，这改变了归纳偏差，从简单函数的形式变为简单函数的形式。

残差映射可以更容易地学习恒等函数，例如将权重层的参数推向零。我们可以通过具有残差块来训练有效的深度神经网络。输入可以通过跨层的残差连接更快地向前传播。因此，我们可以训练更深的网络。例如，原始ResNet论文（He等人，2016）允许使用多达152层。残差网络的另一个好处是，在训练过程中，它允许我们添加初始化为恒等函数的层。毕竟，层的默认行为是让数据不变地通过。在某些情况下，这可以加速非常大型网络的训练。

在残差连接之前，使用门控单元的旁路路径被引入，以有效地训练具有100多层的高速公路网络（Srivastava等人，2015）。使用恒等函数作为旁路路径，ResNet在多个计算机视觉任务上表现出色。残差连接对后续深度神经网络的设计产生了重大影响，无论是卷积还是顺序性质。正如我们稍后将介绍的，Transformer架构（Vaswani等人，2017）采用了残差连接（与其他设计选择一起），并在语言、视觉、语音和强化学习等多样化领域中无处不在。

ResNeXt是一个例子，说明了卷积神经网络设计随着时间的推移是如何演变的：通过更节约地进行计算并将其与激活的大小（通道数）进行权衡，它可以以较低的成本实现更快和更准确的网络。考虑分组卷积的另一种方法是将卷积权重的块对角矩阵。请注意，有相当多的这样的“技巧”可以导致更高效的网络。例如，ShiftNet（Wu等人，2018）通过将移位激活添加到通道中，模拟卷积的效果，这次没有计算成本，提供了增加的功能复杂性。

迄今为止，我们讨论过的设计的一个共同特点是网络设计相当手工，主要依赖于设计者的独创性来找到“正确”的网络超参数。虽然这显然是可行的，但在人类时间上也非常昂贵，而且不能保证结果在任何意义上都是最优的。在第8.8节中，我们将讨论一些更自动化地获得高质量网络的策略。特别是，我们将回顾导致RegNetX/Y模型（Radosavovic等人，2020）的网络设计空间的概念。
'''

'''8_1_deep_convolutional_nerual_networks_alexnet.ipynb
1. Inception block和残差块之间的主要区别：
   - Inception block包含多个不同大小的卷积核并行操作，然后将结果连接起来。这样可以让网络学习不同尺度的特征。Inception block的设计目标是在保持计算效率的同时增加网络的宽度和深度。
   - 残差块包含一个或多个卷积层，其输出与输入相加。这种设计允许网络更容易地学习恒等映射，从而有助于训练更深的网络。残差块的设计目标是通过引入跳跃连接来解决深度网络的梯度消失和表示瓶颈问题。

2. 在计算、准确性和可描述的函数类方面的比较：
   - 计算：Inception block由于其并行结构和多个卷积核的使用，可能具有较高的计算复杂性。而残差块的计算复杂性相对较低，因为它们主要包含常规的卷积层。
   - 准确性：这取决于特定的任务和数据集。在某些情况下，Inception网络可能在处理多尺度特征方面表现更好，而ResNet可能在处理深度模型中的梯度消失问题方面表现更好。
   - 函数类：Inception block和残差块都能描述复杂的函数类。然而，由于残差连接的引入，ResNet可能更容易学习恒等映射，从而有助于训练更深的网络。

3. 为什么我们不能无限制地增加函数的复杂性，即使函数类是嵌套的？
   
   当我们不断增加网络的复杂性时，可能会遇到以下问题：
   - 过拟合：随着网络复杂性的增加，模型可能会对训练数据过度拟合，从而导致在测试数据上的泛化性能下降。
   - 计算资源限制：随着网络复杂性的增加，训练和推理所需的计算资源（如内存和计算能力）也会增加。这可能会导致无法在有限的硬件资源上训练或部署模型。
   - 梯度消失/爆炸：在深度网络中，梯度可能会在反向传播过程中消失或爆炸。这会导致训练困难或不稳定。尽管残差连接等技术可以在一定程度上解决这个问题，但仍然需要在设计网络时权衡复杂性和稳定性。

因此，在实际应用中，我们需要在提高模型性能和控制复杂性之间找到一个平衡点，以实现高效且可泛化的深度学习模型。
'''